In [1]:
import json

from src.helpers import categories_widgets, load_doc_transactions_bankinter, procesar_transacciones, update_categoria
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

In [2]:
file_datos = "movimientos(2).xls"
file_categorias = "categorias.json"
df = load_doc_transactions_bankinter(file_datos)
df.reset_index(drop=True, inplace=True)

In [3]:
comercio_categoria_dict = json.load(open(file_categorias))

In [4]:
# Función para clasificar el comercio
def clasificar_comercio(comercio):
    for key in comercio_categoria_dict:
        if key.lower() in comercio.lower():
            return comercio_categoria_dict[key]
    return 'otros'

# Crear la columna 'Tipo de Comercio' inicial
df['Tipo de Comercio'] = df['COMERCIO/CAJERO'].apply(clasificar_comercio)

# Widgets interactivos para actualizar categorías
tipo_opciones = ['bar', 'restaurante', 'desayuno', 'supermercado','ropa','servicios de internet','vacaciones', 'otros', 'tabaco', 'cosmético', 'muebles','deporte','retirada efectivo','gasolina']
widgets_dict = {}

def actualizar_categoria(change):
    indice = change['owner'].indice
    nuevo_tipo = change['new']
    comercio = df.at[indice, 'COMERCIO/CAJERO']
    comercio_categoria_dict[comercio] = nuevo_tipo
    df['Tipo de Comercio'] = df['COMERCIO/CAJERO'].apply(clasificar_comercio)
    display(df)

for i in range(len(df)):
    dropdown = widgets.Dropdown(options=tipo_opciones, value=df.at[i, 'Tipo de Comercio'])
    dropdown.indice = i
    dropdown.observe(actualizar_categoria, names='value')
    widgets_dict[i] = dropdown

In [5]:
display(widgets.VBox([widgets.HBox([widgets.Label(f"{df.at[i, 'COMERCIO/CAJERO']}:"), widgets_dict[i]]) for i in range(len(df))]))
display(df)

,FECHA,COMERCIO/CAJERO,MOVIMIENTO A,IMPORTE,Tipo de Comercio
0,2024-05-31 00:00:00,COM. USO REDES INTERNACION.,Crédito,-0.56,otros
1,2024-05-31 00:00:00,FARMACIA EL CAMINO,Crédito,-9.52,otros
2,2024-05-31 00:00:00,OPENAI *CHATGPT SUBSCR,Crédito,-18.54,otros
3,2024-06-02 00:00:00,MATILDE PATRIZIA CHIARINI,Crédito,-11.70,otros
4,2024-06-02 00:00:00,AGUERE CULTURAL,Crédito,-18.00,otros
5,2024-06-03 00:00:00,PIZZADILLO,Crédito,-20.00,otros
6,2024-06-02 00:00:00,PAYPAL *GLOVO,Crédito,-27.69,restaurante
7,2024-06-05 00:00:00,FRUTERIA MANOLO QUINTANA,Crédito,-2.73,otros
8,2024-06-05 00:00:00,SUMA,Crédito,-4.40,otros
9,2024-06-06 00:00:00,EL MOMENTO,Crédito,-3.30,desayuno


In [6]:
# guardar cambios en categorías
with open(file_categorias, 'w') as json_file:
    json.dump(comercio_categoria_dict, json_file, indent=4)
print("Diccionario guardado en comercio_categoria_dict.json")

Diccionario guardado en comercio_categoria_dict.json


In [7]:
# Guardar el DataFrame en un archivo Excel, organizando en hojas según el mes y año
df['Mes-Año'] = pd.to_datetime(df['FECHA']).dt.strftime('%Y-%m')
with pd.ExcelWriter('mis_gastos.xlsx', engine='openpyxl') as writer:
    for month_year, group in df.groupby('Mes-Año'):
        group.drop(columns=['Mes-Año'], inplace=True)  # Eliminar la columna auxiliar
        group.to_excel(writer, sheet_name=month_year, index=False)
print("DataFrame guardado en mis_gastos.xlsx")

DataFrame guardado en mis_gastos.xlsx


In [8]:
month_year

'2024-06'